In [ ]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import PIL
import tensorflow as tf
import pathlib
import ipywidgets as widgets
import io
from PIL import Image

from PIL import ImageShow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Download Dataset

In [ ]:
my_data = pd.read_csv("../input/honey-bee-annotated-images/bee_data.csv")
dataset_path = "../input/bee-images-separated/bee_imgs"
data_dir = pathlib.Path(dataset_path)

healthy = list(data_dir.glob('healthy/*'))
PIL.Image.open(str(healthy[0]))

#Use to print out how many photos are in all the photos in the bee_imgs directory
#image_count = len(list(data_dir.glob('*/*.png')))
#print(image_count)

#use to print out how many photos per category in the CSV
#print(my_data['health'].value_counts())

# Create Dataset
Validation split splits the images into 80% for training and 20% for validation

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
#Prints how many classes are in the new dataset
#print(class_names)

# Visualize Dataset
Uncomment to see a sampling of 9 random images that exist in the training data

In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")
#         plt.show() #NEEDED. Will not display images unless this is added.

# Configure Dataset
dataset.cache(): keeps images in memory after they're loaded off the disk for the first epoch
<br>dataset.prefetch(): overlaps data preprocessing and model execution while training

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Standardize Dataset

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

# Create Model
Consists of three convolutional blocks with each having a max pooling layers
<br>The fully connected layer is activated by the ReLU function

In [ ]:
num_classes = 6

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# Compile Data
Use Adam optimizer and the Sparse Categorical Croessentropy loss function

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Model Summary

In [ ]:
model.summary()

# Train Model

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Visualize Training Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predict on New data

Use the commented paths below to check different types of bee png's from the predictions folder. The prediction folder contains different images to how well the data predicts the following:
* Bee drawing
![bee_drawing.jpg](attachment:bee_drawing.jpg)
* Yellow and Black stripes
![bee_pattern.jpg](attachment:bee_pattern.jpg)
* Healthy Bees
![001_054.png](attachment:001_054.png)
![healthy_bee_side_profile.jpg](attachment:healthy_bee_side_profile.jpg)
![healthy_bee_top_view.jpg](attachment:healthy_bee_top_view.jpg)
* Bees with pollen (not being robbed)
![honey_bee_not_robbed_2.jpg](attachment:honey_bee_not_robbed_2.jpg)
![honey_bee_not_robbed_1.jpg](attachment:honey_bee_not_robbed_1.jpg)
* Bees with deformed wings
![unhealthy_bee_2.jpg](attachment:unhealthy_bee_2.jpg)
![unhealthy_bee_1.jpg](attachment:unhealthy_bee_1.jpg)
* Bees visibly infected with varroa mites
![varroa_infected_3.jpg](attachment:varroa_infected_3.jpg)
![varroa_infected_2.jpg](attachment:varroa_infected_2.jpg)
![varroa_infected_1.jpg](attachment:varroa_infected_1.jpg)

In [ ]:
#Bee Drawing
bee_drawing = "../input/new-data-for-predictions/Predictions/bee_drawing.png"

#Yellow and Black Stripes
bee_pattern = "../input/new-data-for-predictions/Predictions/bee_pattern.png"

#Healthy Bees
healthy_bee_from_data  = "../input/bee-images-separated/bee_imgs/healthy/001_054.png"
healthy_bee_side = "../input/new-data-for-predictions/Predictions/healthy_bee_side_profile.png"
healthy_bee_top = "../input/new-data-for-predictions/Predictions/healthy_bee_top_view.png"

#Bees with Pollen
bee_with_pollen_1 = "../input/new-data-for-predictions/Predictions/honey_bee_not_robbed_1.png"
bee_with_pollen_2 = "../input/new-data-for-predictions/Predictions/honey_bee_not_robbed_2.png"

#Bees with Deformed Wings
unhealthy_bee_1 = "../input/new-data-for-predictions/Predictions/unhealthy_bee_1.png"
unhealthy_bee_2 = "../input/new-data-for-predictions/Predictions/unhealthy_bee_2.png"

#Bees Infected with Varroa Mites
varroa_infected_1 = "../input/new-data-for-predictions/Predictions/varroa_infected_1.png"
varroa_infected_2 = "../input/new-data-for-predictions/Predictions/varroa_infected_2.png"
varroa_infected_3 = "../input/new-data-for-predictions/Predictions/varroa_infected_3.png"

In [ ]:
#Replace the file path with any from the previous section and run the section below to see the results
bee_img = "../input/new-data-for-predictions/Predictions/honey_bee_not_robbed_1.png"

In [ ]:
#For pictures off the internet
#_url = "https://"
#_path = tf.keras.utils.get_file('File_name', origin=_url)


img = keras.preprocessing.image.load_img(
    bee_img, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to the category {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

# Predictions Results:
 <br>bee_drawing = **ants** with a **99.83** percent confidence.
 <br><br>bee_pattern = **robbed** with a **100.00** percent confidence.
 <br><br>healthy_bee_from_data = **healthy** with a **99.95** percent confidence.
 <br><br>healthy_bee_side = **ants** with a **90.64** percent confidence.
 <br><br>healthy_bee_top = **healthy** with a **99.84** percent confidence.
 <br><br>bee_with_pollen_1 = **ants** with a **99.08** percent confidence.
 <br><br>bee_with_pollen_2 = **few_var** with a **88.72** percent confidence.
 <br><br>unhealthy_bee_1 = **ants** with a **100.00** percent confidence.
 <br><br>unhealthy_bee_2= **robbed** with a **99.99** percent confidence.
 <br><br>varroa_infected_1 = **healthy** with a **100.00** percent confidence.
 <br><br>varroa_infected_2 = **healthy** with a **100.00** percent confidence.
 <br><br>varroa_infected_3 =**ants** with a **100.00** percent confidence.